In [ ]:
#!pip install tensorflow-gpu==2.3

In [1]:
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.layers import Layer
from tensorflow.keras import initializers, regularizers, constraints
import six
from tensorflow.keras.utils import deserialize_keras_object
from joblib import dump, load

In [2]:
from google.colab import files
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Dense,Input,GRU,Embedding,Flatten
from tensorflow.keras.models import Model
import keras.backend as K
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from tensorflow.keras.layers import Input, Dense, Conv2D, BatchNormalization, Activation, Dropout, GRU, Embedding
from tensorflow.keras.models import Model
from tensorflow.keras import activations
from tensorflow.keras.layers import Layer
from tensorflow.keras import layers
import tensorflow as tf
 
from tensorflow.keras.layers import concatenate, Lambda

In [3]:
tf.test.gpu_device_name()

'/device:GPU:0'

In [4]:

ENCODER_SEQ_LEN = 38
DECODER_SEQ_LEN = 39
EN_VOCAB_SIZE = len(load('/content/encoder_vocab_lower.joblib'))+1
DE_VOCAB_SIZE=len(load('/content/decoder_vocab_lower.joblib'))+1
units = 100
x = load('/content/encoder_train.joblib')
y = load('/content/decoder_inp_train.joblib')
z = load('/content/decoder_out_train.joblib')
encoder_test=load('/content/encoder_test.joblib')
decoder_inp_test=load('/content/decoder_inp_test.joblib')
decoder_out_test=load('/content/decoder_out_test.joblib')
decoder_embedding=load('/content/decoder_embedding2.joblib')
print(decoder_embedding.shape)
encoder_embedding=load('/content/encoder_embedding1.joblib')
print(encoder_embedding.shape)

(3040, 300)
(3702, 300)


In [5]:
def serialize(activation):
    '''Get the name of activation'''
    return activation.__name__
def get(identifier):
    """Get the `identifier` activation function.
    # Arguments
        identifier: None or str, name of the function.
    # Returns
        The activation function, `linear` if `identifier` is None.
    # Raises
        ValueError if unknown identifier
    """
    if isinstance(identifier, six.string_types):
        identifier = str(identifier)
        return deserialize(identifier)
    elif callable(identifier):
        if isinstance(identifier, Layer):
            warnings.warn(
                'Do not pass a layer instance (such as {identifier}) as the '
                'activation argument of another layer. Instead, advanced '
                'activation layers should be used just like any other '
                'layer in a model.'.format(
                    identifier=identifier.__class__.__name__))
        return identifier
    else:
        raise ValueError('Could not interpret '
                         'activation function identifier:', identifier) 
        
def _monotonic_attetion(probabilities, attention_prev, mode):

    """Compute monotonic attention distribution from choosing probabilities.
    Implemented Based on -
    https://colinraffel.com/blog/online-and-linear-time-attention-by-enforcing-monotonic-alignments.html
    https://arxiv.org/pdf/1704.00784.pdf
    Mainly implemented by referring
    https://github.com/craffel/mad/blob/b3687a70615044359c8acc440e43a5e23dc58309/example_decoder.py#L22
    # Arguments:
        probabilities: Probability of choosing input sequence..
                       Should be of shape (batch_size, max_length),
                       and should all be in the range [0, 1].
        attention_prev: The attention distribution from the previous output timestep.
                            Should be of shape (batch_size, max_length).
                            For the first output timestep,
                            should be [1, 0, 0, ...,0] for all n in [0, ... batch_size - 1].
        mode: How to compute the attention distribution.
              Must be one of 'recursive', 'parallel', or 'hard'.
              - 'recursive' uses tf.scan to recursively compute the distribution.
              This is slowest but is exact, general, and does not suffer from
              numerical instabilities.
              - 'parallel' uses parallelized cumulative-sum and cumulative-product
              operations to compute a closed-form solution to the recurrence relation
              defining the attention distribution.  This makes it more efficient than 'recursive',
              but it requires numerical checks which make the distribution non-exact.
              This can be a problem in particular when max_length is long and/or
              probabilities has entries very close to 0 or 1.
              - 'hard' requires that  the probabilities in p_choose_i are all either 0 or 1,
              and subsequently uses a more efficient and exact solution.
    # Returns: A tensor of shape (batch_size, max_length) representing the attention distributions
               for each sequence in the batch.
    # Raises:
             ValueError: if mode is not one of 'recursive', 'parallel', 'hard'."""
    if mode == 'hard':
        #Remove any probabilities before the index chosen last time step
        probabilities = probabilities*tf.cumsum(attention_prev, axis=1)
        attention = probabilities*tf.math.cumprod(1-probabilities, axis=1, exclusive=True)
    elif mode == 'recursive':
        batch_size = tf.shape(input=probabilities)[0]
        shifted_1mp_probabilities = tf.concat([tf.ones((batch_size, 1)),\
            1 - probabilities[:, :-1]], 1)
        attention = probabilities*tf.transpose(a=tf.scan(lambda x, yz: tf.reshape(yz[0]*x + yz[1],\
            (batch_size,)), [tf.transpose(a=shifted_1mp_probabilities),\
                tf.transpose(a=attention_prev)], tf.zeros((batch_size,))))
    elif mode == 'parallel':
        cumprod_1mp_probabilities = tf.exp(tf.cumsum(tf.math.log(tf.clip_by_value(1-probabilities,\
            1e-10, 1)), axis=1, exclusive=True))
        attention = probabilities*cumprod_1mp_probabilities*tf.cumsum(attention_prev/\
            tf.clip_by_value(cumprod_1mp_probabilities, 1e-10, 1.), axis=1)
    else:
        raise ValueError("Mode must be 'hard', 'parallel' or 'recursive' ")

    return attention 
def deserialize(name, custom_objects=None):
    '''deserialize Keras Object'''
    return deserialize_keras_object(
        name,
        module_objects=globals(),
        custom_objects=custom_objects,
        printable_module_name='activation function')    
def softmax(inputs, axis=-1):
    '''softmax activation function
    # Arguments
        inputs = Input Tensor.
        axis = Integer, axis along which the softmax normalization is applied.
    # Returns
        Tensor, output of the softmax transformation
    '''
    return tf.nn.softmax(inputs, axis=axis)     
def _attention_score(dec_ht,
                     enc_hs,
                     attention_type,
                     weightwa=None,
                     weightua=None,
                     weightva=None):
    if attention_type == 'bahdanau':
        score = weightva(tf.nn.tanh(weightwa(dec_ht) + weightua(enc_hs)))
        score = tf.squeeze(score, [2])
    elif attention_type == 'dot':
        score = tf.matmul(dec_ht, enc_hs, transpose_b=True)
        score = tf.squeeze(score, 1)
    elif attention_type == 'general':
        score = weightwa(enc_hs)
        score = tf.matmul(dec_ht, score, transpose_b=True)
        score = tf.squeeze(score, 1)
    elif attention_type == 'concat':
        
        dec_ht = tf.tile(dec_ht, [1, enc_hs.shape[1], 1])
        score = weightva(tf.nn.tanh(weightwa(tf.concat((dec_ht, enc_hs), axis=-1))))
        score = tf.squeeze(score, 2)
    return score            

In [6]:
class BahdanauAttention(Layer):
    '''
    BahdanauAttention:
    Implemented based on below paper
        https://arxiv.org/pdf/1409.0473.pdf
        attention_weights = probability_fn((Va * tanh(Wa*Ht+Ua*Hs+b))/sqrt(scaling_factor))
    # Arguments
        units = number of hidden units to use.
        probability_fn = probability function to get probabilities(weights for attention)
                         You can use 'softmax' or 'hardmax' or 'sparsemax' or any custom
                         function which takes input distribution and returns probability dist.
        dropout_rate = dropout for attention weights (between 0 and 1, 0 - no dropout).
        return_aweights = Bool, whether to return attention weights or not.
        scaling_factor = int/float to scale the score vector. default None=1
        weights_initializer = initializer for weight matrix
        bias_initializer = initializer for bias values
        weights_regularizer = Regularize the weights (U, W, V)
        bias_regularizer = Regularize the bias (b)
        weights_constraint = Constraint function applied to the weights
        bias_constraint = Constraint function applied to the bias
    # Returns
        context_vector = context vector after applying attention.
        attention_weights = attention weights only if `return_aweights=True`.
    # Inputs to the layer
        inputs = dictionary with keys "enocderHs", "decoderHt".
                enocderHs = all the encoder hidden states,
                            shape - (Batchsize, encoder_seq_len, enc_hidden_size)
                 decoderHt = hidden state of decoder at that timestep,
                            shape - (Batchsize, dec_hidden_size)
        mask = You can apply mask for padded values or any custom values
               while calculating attention.
               if you are giving mask for encoder and deocoder then you have
               to give a dict similar to inputs. (keys: enocderHs, decoderHt)
               else you can give only for enocoder normally.(one tensor)
               mask shape should be (Batchsize, encoder_seq_len)
    '''
    def __init__(self, units,
                 probability_fn='softmax',
                 dropout_rate=0,
                 return_aweights=False,
                 scaling_factor=None,
                 weights_initializer='he_normal',
                 bias_initializer='zeros',
                 weights_regularizer=None,
                 bias_regularizer=None,
                 weights_constraint=None,
                 bias_constraint=None,
                 **kwargs):

        if 'name' not in kwargs:
            kwargs['name'] = ""
        super(BahdanauAttention, self).__init__(**kwargs)

        self.units = units
        self.dropout_rate = dropout_rate
        self.return_aweights = return_aweights
        self.scaling_factor = scaling_factor
        self.probability_fn = get(probability_fn)
        self.weights_initializer = initializers.get(weights_initializer)
        self.bias_initializer = initializers.get(bias_initializer)
        self.weights_regularizer = regularizers.get(weights_regularizer)
        self.bias_regularizer = regularizers.get(bias_regularizer)
        self.weights_constraint = constraints.get(weights_constraint)
        self.bias_constraint = constraints.get(bias_constraint)
        self._wa = layers.Dense(self.units, use_bias=False,\
            kernel_initializer=weights_initializer, bias_initializer=bias_initializer,\
                kernel_regularizer=weights_regularizer, bias_regularizer=bias_regularizer,\
                    kernel_constraint=weights_constraint, bias_constraint=bias_constraint,\
                        name=self.name+"Wa")
        self._ua = layers.Dense(self.units,\
            kernel_initializer=weights_initializer, bias_initializer=bias_initializer,\
                kernel_regularizer=weights_regularizer, bias_regularizer=bias_regularizer,\
                    kernel_constraint=weights_constraint, bias_constraint=bias_constraint,\
                        name=self.name+"Ua")
        self._va = layers.Dense(1, use_bias=False, kernel_initializer=weights_initializer,\
            kernel_regularizer=weights_regularizer, bias_regularizer=bias_regularizer,\
                bias_initializer=bias_initializer, kernel_constraint=weights_constraint,\
                    bias_constraint=bias_constraint, name=self.name+"Va")
        self.supports_masking = True


    def build(self, input_shape):
        '''build'''
        assert isinstance(input_shape, dict)

        shape_en, shape_dc = input_shape['enocderHs'], input_shape['decoderHt']

        assert len(shape_en) >= 3, "Encoder Hiddenstates/output should be 3 dim or more \
        ( B x T x H ), but got {} dim".format(len(shape_en))

        assert len(shape_dc) == 2, "Decoder Hidden/output should be 2 \
        dim (B x H), but got {} dim".format(len(shape_dc))

        self.built = True # pylint: disable=W0201
    def call(self, inputs, mask=None):
        '''call'''
        assert isinstance(inputs, dict)

        if ('enocderHs' not in inputs.keys())or ('decoderHt' not in inputs.keys()):
            raise ValueError("Input to the layer must be a dict with \
            keys=['enocderHs','decoderHt']")
        if isinstance(mask, dict):
            mask_enc = mask.get('enocderHs', None)
            mask_dec = mask.get('decoderHt', None)
        else:
            mask_enc = mask
            mask_dec = None

        enc_out, dec_prev_hs = tf.cast(inputs['enocderHs'], tf.float32), \
            tf.cast(inputs['decoderHt'], tf.float32)
        if mask_dec is not None:
            dec_prev_hs = dec_prev_hs * tf.cast(mask_dec, dec_prev_hs.dtype)
        if mask_enc is not None:
            enc_out = enc_out * tf.cast(tf.expand_dims(mask_enc, 2), enc_out.dtype)

        # decprev_hs - Decoder hidden shape == (batch_size, hidden size)
        # hidden_with_time_axis shape == (batch_size, 1, hidden size)
        dec_hidden_with_time_axis = tf.expand_dims(dec_prev_hs, 1)

        # score shape == (batch_size, max_length)
        score = _attention_score(dec_ht=dec_hidden_with_time_axis, enc_hs=enc_out,\
                    attention_type='concat', weightwa=self._wa,\
                        weightua=self._ua, weightva=self._va)

        if self.scaling_factor is not None:
            score = score/tf.sqrt(self.scaling_factor)

        #if mask_enc is not None:
            #score = score + (tf.cast(tf.math.equal(mask_enc, False), score.dtype)*-1e9)

        # attention_weights shape == (batch_size, max_length)
        attention_weights = self.probability_fn(score, axis=-1)
        #(batch_size, 1, max_length)
        attention_weights = tf.expand_dims(attention_weights, 1)

        if self.dropout_rate != 0:
            attention_weights = tf.nn.dropout(x=attention_weights, rate=self.dropout_rate)

        #context_vector shape (batch_size, hidden_size)
        context_vector = tf.matmul(attention_weights, enc_out)
        context_vector = tf.squeeze(context_vector, 1, name="context_vector")

        if self.return_aweights:
            return context_vector, tf.squeeze(attention_weights, 1, name='attention_weights')
        return context_vector


In [7]:
class MonotonicBahdanauAttention(Layer):
    '''
    MonotonicBahdanauAttention
    Implemented based on below paper
        https://arxiv.org/pdf/1704.00784.pdf
    # Arguments
        units = number of hidden units to use.
        mode = How to compute the attention distribution.
              Must be one of 'recursive', 'parallel', or 'hard'.
              - 'recursive' uses tf.scan to recursively compute the distribution.
              This is slowest but is exact, general, and does not suffer from
              numerical instabilities.
              - 'parallel' uses parallelized cumulative-sum and cumulative-product
              operations to compute a closed-form solution to the recurrence relation
              defining the attention distribution.  This makes it more efficient than 'recursive',
              but it requires numerical checks which make the distribution non-exact.
              This can be a problem in particular when max_length is long and/or
              probabilities has entries very close to 0 or 1.
              - 'hard' requires that  the probabilities in p_choose_i are all either 0 or 1,
              and subsequently uses a more efficient and exact solution.
        return_aweights = Bool, whether to return attention weights or not.
        scaling_factor = int/float to scale the score vector. default None=1
        noise_std = standard deviation of noise which will be added before
                    applying sigmoid function.(pre-sigmoid noise). If it is 0 or
                    mode="hard", we won't add any noise.
        weights_initializer = initializer for weight matrix
        weights_regularizer = Regularize the weights
        weights_constraint = Constraint function applied to the weights
    # Returns
        context_vector = context vector after applying attention.
        attention_weights = attention weights only if `return_aweights=True`.
    # Inputs to the layer
        inputs = dictionary with keys "enocderHs", "decoderHt", "prevAttention".
                enocderHs = all the encoder hidden states,
                            shape - (Batchsize, encoder_seq_len, enc_hidden_size)
                decoderHt = hidden state of decoder at that timestep,
                            shape - (Batchsize, dec_hidden_size)
                prevAttention = Previous probability distribution of attention
                                (previous attention weights)
        mask = You can apply mask for padded values or any custom values
               while calculating attention.
               if you are giving mask for encoder and deocoder then you have
               to give a dict similar to inputs. (keys: enocderHs, decoderHt)
               else you can give only for enocoder normally.(one tensor)
               mask shape should be (Batchsize, encoder_seq_len)
    '''
 
    def __init__(self, units,
                 mode='parallel',
                 return_aweights=False,
                 scaling_factor=None,
                 noise_std=0,
                 weights_initializer='he_normal',
                 bias_initializer='zeros',
                 weights_regularizer=None,
                 bias_regularizer=None,
                 weights_constraint=None,
                 bias_constraint=None,
                 **kwargs):
        if 'name' not in kwargs:
            kwargs['name'] = ""
        super(MonotonicBahdanauAttention, self).__init__(**kwargs)
        self.units = units
        self.mode = mode
        self.return_aweights = return_aweights
        self.scaling_factor = scaling_factor
        self.noise_std = noise_std
        self.weights_initializer = initializers.get(weights_initializer)
        self.bias_initializer = initializers.get(bias_initializer)
        self.weights_regularizer = regularizers.get(weights_regularizer)
        self.bias_regularizer = regularizers.get(bias_regularizer)
        self.weights_constraint = constraints.get(weights_constraint)
        self.bias_constraint = constraints.get(bias_constraint)
        self._wa = layers.Dense(self.units, use_bias=False,\
            kernel_initializer=weights_initializer, bias_initializer=bias_initializer,\
                kernel_regularizer=weights_regularizer, bias_regularizer=bias_regularizer,\
                    kernel_constraint=weights_constraint, bias_constraint=bias_constraint,\
                        name=self.name+"Wa")
        self._ua = layers.Dense(self.units,\
            kernel_initializer=weights_initializer, bias_initializer=bias_initializer,\
                kernel_regularizer=weights_regularizer, bias_regularizer=bias_regularizer,\
                    kernel_constraint=weights_constraint, bias_constraint=bias_constraint,\
                        name=self.name+"Ua")
        self._va = layers.Dense(1, use_bias=False, kernel_initializer=weights_initializer,\
            kernel_regularizer=weights_regularizer, bias_regularizer=bias_regularizer,\
                bias_initializer=bias_initializer, kernel_constraint=weights_constraint,\
                    bias_constraint=bias_constraint, name=self.name+"Va")
        self.supports_masking = True
 
    def build(self, input_shape):
        '''build'''
        assert isinstance(input_shape, dict)
 
        shape_en, shape_dc = input_shape['enocderHs'], input_shape['decoderHt']
        shape_prob = input_shape['prevAttention']
        
 
        assert len(shape_en) >= 3, "Encoder Hiddenstates/output should be 3 dim or more \
        ( B x T x H ), but got {} dim".format(len(shape_en))
 
        assert len(shape_dc) == 2, "Decoder Hidden/output should be 2 \
        dim (B x H), but got {} dim".format(len(shape_dc))
 
        assert len(shape_prob) == 2, "Previous probability should be 2 \
        dim (B x H), but got {} dim".format(len(shape_prob))
 
        self.built = True # pylint: disable=W0201
 
    def call(self, inputs, mask=None, training=True):
        '''call'''
        assert isinstance(inputs, dict)
       
 
        if ('enocderHs' not in inputs.keys())or ('decoderHt' not in inputs.keys()\
            or 'prevAttention' not in inputs.keys()):
            raise ValueError("Input to the layer must be a dict with \
            keys=['enocderHs','decoderHt', 'prevAttention']")
 
        if isinstance(mask, dict):
            mask_enc = mask.get('enocderHs', None)
            mask_dec = mask.get('decoderHt', None)
        else:
            mask_enc = mask
            mask_dec = None
        enc_out, dec_prev_hs = tf.cast(inputs['enocderHs'], tf.float32), \
            tf.cast(inputs['decoderHt'], tf.float32)
 
        prev_attention = inputs['prevAttention']
      
 
        if mask_dec is not None:
            dec_prev_hs = dec_prev_hs * tf.cast(mask_dec, dec_prev_hs.dtype)
        if mask_enc is not None:
            enc_out = enc_out * tf.cast(tf.expand_dims(mask_enc, 2), enc_out.dtype)
 
        # decprev_hs - Decoder hidden shape == (batch_size, hidden size)
        # hidden_with_time_axis shape == (batch_size, 1, hidden size)
        dec_hidden_with_time_axis = tf.expand_dims(dec_prev_hs, 1)
       
 
        # score shape == (batch_size, max_length)
        score = _attention_score(dec_ht=dec_hidden_with_time_axis, enc_hs=enc_out,\
                    attention_type='concat', weightwa=self._wa,\
                        weightua=self._ua, weightva=self._va)
 
        if self.scaling_factor is not None:
            score = score/tf.sqrt(self.scaling_factor)
 
        if training:
            if self.noise_std > 0:
                random_noise = tf.random.normal(shape=tf.shape(input=score), mean=0,\
                    stddev=self.noise_std, dtype=score.dtype, seed=self.seed)
                score = score + random_noise
 
        if mask_enc is not None:
            score = score + (tf.cast(tf.math.equal(mask_enc, False), score.dtype)*-1e9)
 
        if self.mode == 'hard':
            probabilities = tf.cast(score > 0, score.dtype)
        else:
            probabilities = tf.sigmoid(score)
 
        attention_weights = _monotonic_attetion(probabilities, prev_attention, self.mode)
        attention_weights = tf.expand_dims(attention_weights, 1)
 
        #context_vector shape (batch_size, hidden_size)
        context_vector = tf.matmul(attention_weights, enc_out)
        context_vector = tf.squeeze(context_vector, 1, name="context_vector")
 
        if self.return_aweights:
         
            return context_vector, tf.squeeze(attention_weights, 1, name='attention_weights')
        return context_vector
 
    def compute_output_shape(self, input_shape):
        '''compute output shape'''
        assert isinstance(input_shape, dict)
        shape_en = input_shape['enocderHs']
        if self.return_aweights:
            output_shape = [(shape_en[0], shape_en[2]), (shape_en[0], shape_en[1])]
            return output_shape
        output_shape = shape_en[0], shape_en[2]
        return output_shape
 
    def get_config(self):
        '''Config'''
        config = {'units': self.units,
                  'mode':self.mode,
                  'return_aweights' : self.return_aweights,
                  'scaling_factor' : self.scaling_factor,
                  'noise_std':self.noise_std,
                  'weights_initializer' : initializers.serialize(self.weights_initializer),
                  'bias_initializer' : initializers.serialize(self.bias_initializer),
                  'weights_regularizer' : regularizers.serialize(self.weights_regularizer),
                  'bias_regularizer' : regularizers.serialize(self.bias_regularizer),
                  'weights_constraint' : constraints.serialize(self.weights_constraint),
                  'bias_constraint': constraints.serialize(self.bias_constraint)}
        base_config = super(MonotonicBahdanauAttention, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))

In [13]:
def encoder():
    input_encoder = Input(shape=(ENCODER_SEQ_LEN,), name='encoder_input')

    x_embedded = Embedding(input_dim=EN_VOCAB_SIZE, output_dim=300, input_length=ENCODER_SEQ_LEN,
                           mask_zero=False, name="embedding_layer_encoder",weights=[encoder_embedding])(input_encoder)
    print(x_embedded.shape)
    gru_output, gru_state= GRU(units, return_state=True, return_sequences=True, name="Encoder_GRU")(x_embedded)
    return Model(inputs=input_encoder, outputs=[gru_output, gru_state])

In [14]:
 
def OneStepDecoder():
    
    #inputs
    input_decoder = Input(shape=(1,),name="dec_input")
    
    input_state = Input(shape=(units,),name="Dec_states")
    
    encoder_outputs = Input(shape=(ENCODER_SEQ_LEN, units,),name="EncoderStates")
    prevAttention=Input(shape=(38,),name="PreviousAttention")
    
    #mask_encoder = Input(shape=(ENCODER_SEQ_LEN,), name='mask_encoder')
    
    # Embedding layers
    dec_embdd = Embedding(input_dim=DE_VOCAB_SIZE, output_dim=300,
                      input_length=1,mask_zero=False, name="Decoder_Embedding_layer",weights=[decoder_embedding])(input_decoder)
    
    
    z_attention = dict()
    z_attention['enocderHs'] = encoder_outputs
    z_attention['decoderHt'] = input_state
    z_attention['prevAttention']=prevAttention
    print(z_attention['prevAttention'].shape)
    
    context_vector,prev_Attention = MonotonicBahdanauAttention(units=20, name='parallel',return_aweights=True)(z_attention)
    print(prevAttention)
    
    concat = concatenate([tf.cast(tf.expand_dims(context_vector, 1), dec_embdd.dtype), dec_embdd],name="concat")
    
    decoder_output, Decoder_state = GRU(units=units,return_state=True,name="DecGRU")(concat, initial_state=input_state)
    
    output = Dense(units=DE_VOCAB_SIZE,activation="softmax",name="DenseOut")(decoder_output)
    print(output.shape)
    return Model(inputs=[input_decoder, input_state, encoder_outputs,prevAttention],outputs=[output, Decoder_state,prev_Attention],name="OneStepDecoder")

In [10]:
 
def encoder_decoder():
    
    encoder_input = Input(shape=(ENCODER_SEQ_LEN,), name='encoder_input_final')
    decoder_input = Input(shape=(DECODER_SEQ_LEN,), name="Decoder_input_final")
    
    enc = encoder()
 
    outputs_encoder, encoder_state = enc(encoder_input)
    print(outputs_encoder.shape)
    #batch size
    batch_size=4
    max_seq=38
    
    all_outputs= []
    #intialisation of numpy zero matrix for previous attention
    prevAttention=np.zeros((batch_size,max_seq))
    #print(prevAttention.shape)
 
    prevAttention[:,0]=1
    decoder_one_att = OneStepDecoder()
 
    
    
    print(prevAttention)
    
    for timestep in range(DECODER_SEQ_LEN):
        
        inputs = Lambda(lambda x: x[:,timestep:timestep+1])(decoder_input)
        
        output, encoder_state,prevAttention = decoder_one_att([inputs, encoder_state, outputs_encoder,prevAttention])
        
        output = Lambda(lambda x:  tf.expand_dims(x, 1))(output)
        
        all_outputs.append(output)
        
  
    #decoder_outputs = Lambda(lambda x: tf.keras.backend.concatenate(all_outputs,1))(all_outputs)
    decoder_outputs=tf.concat(all_outputs,1)
    print(decoder_outputs.shape)
    
    return Model(inputs=[encoder_input, decoder_input], outputs=decoder_outputs)

In [15]:
EC=encoder_decoder()

(None, 38, 300)
(None, 38, 100)
(None, 38)
KerasTensor(type_spec=TensorSpec(shape=(None, 38), dtype=tf.float32, name='PreviousAttention'), name='PreviousAttention', description="created by layer 'PreviousAttention'")
(None, 3040)
[[1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]
(4, 39, 3040)


In [ ]:
EC.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
Decoder_input_final (InputLayer [(None, 39)]         0                                            
__________________________________________________________________________________________________
encoder_input_final (InputLayer [(None, 38)]         0                                            
__________________________________________________________________________________________________
lambda (Lambda)                 (None, 1)            0           Decoder_input_final[0][0]        
__________________________________________________________________________________________________
model (Functional)              [(None, 38, 100), (N 1231200     encoder_input_final[0][0]        
____________________________________________________________________________________________

With Mask Values.

In [12]:
EC.compile(optimizer=tf.keras.optimizers.Adam(),loss=tf.keras.losses.sparse_categorical_crossentropy)
EC.fit(x=[x,y],y=z,epochs=50,verbose=1,batch_size=4,validation_data=([encoder_test,decoder_inp_test],decoder_out_test))

Epoch 1/50
492/492 [==============================] - 216s 205ms/step - loss: 7.8517 - val_loss: 7.3894
Epoch 2/50
492/492 [==============================] - 72s 147ms/step - loss: 7.2290 - val_loss: 6.8247
Epoch 3/50
492/492 [==============================] - 72s 146ms/step - loss: 6.6621 - val_loss: 6.2943
Epoch 4/50
492/492 [==============================] - 72s 146ms/step - loss: 6.1624 - val_loss: 5.7982
Epoch 5/50
492/492 [==============================] - 72s 146ms/step - loss: 5.6633 - val_loss: 5.3351
Epoch 6/50
492/492 [==============================] - 71s 145ms/step - loss: 5.1964 - val_loss: 4.9063
Epoch 7/50
492/492 [==============================] - 72s 146ms/step - loss: 4.7812 - val_loss: 4.5128
Epoch 8/50
492/492 [==============================] - 72s 145ms/step - loss: 4.3572 - val_loss: 4.1550
Epoch 9/50
492/492 [==============================] - 71s 144ms/step - loss: 4.0458 - val_loss: 3.8386
Epoch 10/50
492/492 [==============================] - 71s 145ms/step - 

Without using Mask Values.

In [16]:
EC.compile(optimizer=tf.keras.optimizers.Adam(),loss=tf.keras.losses.sparse_categorical_crossentropy)
EC.fit(x=[x,y],y=z,epochs=50,verbose=1,batch_size=4,validation_data=([encoder_test,decoder_inp_test],decoder_out_test))

Epoch 1/50
492/492 [==============================] - 93s 104ms/step - loss: 3.6321 - val_loss: 2.3694
Epoch 2/50
492/492 [==============================] - 41s 83ms/step - loss: 2.2554 - val_loss: 2.3338
Epoch 3/50
492/492 [==============================] - 41s 84ms/step - loss: 2.2941 - val_loss: 2.3041
Epoch 4/50
492/492 [==============================] - 42s 85ms/step - loss: 2.1867 - val_loss: 2.2633
Epoch 5/50
492/492 [==============================] - 41s 84ms/step - loss: 2.1321 - val_loss: 2.2186
Epoch 6/50
492/492 [==============================] - 41s 84ms/step - loss: 2.0311 - val_loss: 2.1717
Epoch 7/50
492/492 [==============================] - 41s 84ms/step - loss: 1.9107 - val_loss: 2.1742
Epoch 8/50
492/492 [==============================] - 41s 84ms/step - loss: 1.8903 - val_loss: 2.1365
Epoch 9/50
492/492 [==============================] - 41s 84ms/step - loss: 1.8102 - val_loss: 2.1273
Epoch 10/50
492/492 [==============================] - 41s 84ms/step - loss: 1.72

KeyboardInterrupt: ignored

In [ ]:
EC.compile(optimizer=tf.keras.optimizers.Adam(0.01),loss=tf.keras.losses.sparse_categorical_crossentropy)
EC.fit(x=[x,y],y=z,epochs=260,verbose=1,batch_size=4,validation_data=([encoder_test,decoder_inp_test],decoder_out_test))

Epoch 1/260
492/492 [==============================] - 103s 113ms/step - loss: 3.0869 - val_loss: 2.5900
Epoch 2/260
492/492 [==============================] - 44s 90ms/step - loss: 2.4069 - val_loss: 2.4948
Epoch 3/260
492/492 [==============================] - 44s 90ms/step - loss: 2.4217 - val_loss: 2.4935
Epoch 4/260
492/492 [==============================] - 44s 90ms/step - loss: 2.3155 - val_loss: 2.4923
Epoch 5/260
492/492 [==============================] - 44s 90ms/step - loss: 2.1091 - val_loss: 2.5047
Epoch 6/260
492/492 [==============================] - 44s 89ms/step - loss: 2.0723 - val_loss: 2.4770
Epoch 7/260
492/492 [==============================] - 44s 90ms/step - loss: 2.0307 - val_loss: 2.4883
Epoch 8/260
492/492 [==============================] - 44s 89ms/step - loss: 1.8822 - val_loss: 2.5922
Epoch 9/260
492/492 [==============================] - 44s 89ms/step - loss: 1.9657 - val_loss: 2.6755
Epoch 10/260
 52/492 [==>...........................] - ETA: 39s - loss

KeyboardInterrupt: ignored

In [ ]:
!pip install tensorflow_addons

     |████████████████████████████████| 706kB 9.0MB/s 


In [ ]:
import tensorflow_addons as tfa
adamW= tfa.optimizers.AdamW(
    weight_decay=0.8,
    learning_rate = 0.001,
    beta_1 = 0.9,
    beta_2 = 0.999,
    epsilon = 1e-07,
    amsgrad = False,
    name= 'AdamW'   
)

In [ ]:
#Reference:https://github.com/bckenstler/CLR/blob/master/clr_callback.py
from tensorflow.keras.callbacks import *
from tensorflow.keras import backend as K
import numpy as np

class CyclicLR(Callback):
    """This callback implements a cyclical learning rate policy (CLR).
    The method cycles the learning rate between two boundaries with
    some constant frequency, as detailed in this paper (https://arxiv.org/abs/1506.01186).
    The amplitude of the cycle can be scaled on a per-iteration or 
    per-cycle basis.
    This class has three built-in policies, as put forth in the paper.
    "triangular":
        A basic triangular cycle w/ no amplitude scaling.
    "triangular2":
        A basic triangular cycle that scales initial amplitude by half each cycle.
    "exp_range":
        A cycle that scales initial amplitude by gamma**(cycle iterations) at each 
        cycle iteration.
    For more detail, please see paper.
    
    # Example
        ```python
            clr = CyclicLR(base_lr=0.001, max_lr=0.006,
                                step_size=2000., mode='triangular')
            model.fit(X_train, Y_train, callbacks=[clr])
        ```
    
    Class also supports custom scaling functions:
        ```python
            clr_fn = lambda x: 0.5*(1+np.sin(x*np.pi/2.))
            clr = CyclicLR(base_lr=0.001, max_lr=0.006,
                                step_size=2000., scale_fn=clr_fn,
                                scale_mode='cycle')
            model.fit(X_train, Y_train, callbacks=[clr])
        ```    
    # Arguments
        base_lr: initial learning rate which is the
            lower boundary in the cycle.
        max_lr: upper boundary in the cycle. Functionally,
            it defines the cycle amplitude (max_lr - base_lr).
            The lr at any cycle is the sum of base_lr
            and some scaling of the amplitude; therefore 
            max_lr may not actually be reached depending on
            scaling function.
        step_size: number of training iterations per
            half cycle. Authors suggest setting step_size
            2-8 x training iterations in epoch.
        mode: one of {triangular, triangular2, exp_range}.
            Default 'triangular'.
            Values correspond to policies detailed above.
            If scale_fn is not None, this argument is ignored.
        gamma: constant in 'exp_range' scaling function:
            gamma**(cycle iterations)
        scale_fn: Custom scaling policy defined by a single
            argument lambda function, where 
            0 <= scale_fn(x) <= 1 for all x >= 0.
            mode paramater is ignored 
        scale_mode: {'cycle', 'iterations'}.
            Defines whether scale_fn is evaluated on 
            cycle number or cycle iterations (training
            iterations since start of cycle). Default is 'cycle'.
    """

    def __init__(self, base_lr=0.001, max_lr=0.006, step_size=2000., mode='triangular',
                 gamma=1., scale_fn=None, scale_mode='cycle'):
        super(CyclicLR, self).__init__()

        self.base_lr = base_lr
        self.max_lr = max_lr
        self.step_size = step_size
        self.mode = mode
        self.gamma = gamma
        if scale_fn == None:
            if self.mode == 'triangular':
                self.scale_fn = lambda x: 1.
                self.scale_mode = 'cycle'
            elif self.mode == 'triangular2':
                self.scale_fn = lambda x: 1/(2.**(x-1))
                self.scale_mode = 'cycle'
            elif self.mode == 'exp_range':
                self.scale_fn = lambda x: gamma**(x)
                self.scale_mode = 'iterations'
        else:
            self.scale_fn = scale_fn
            self.scale_mode = scale_mode
        self.clr_iterations = 0.
        self.trn_iterations = 0.
        self.history = {}

        self._reset()

    def _reset(self, new_base_lr=None, new_max_lr=None,
               new_step_size=None):
        """Resets cycle iterations.
        Optional boundary/step size adjustment.
        """
        if new_base_lr != None:
            self.base_lr = new_base_lr
        if new_max_lr != None:
            self.max_lr = new_max_lr
        if new_step_size != None:
            self.step_size = new_step_size
        self.clr_iterations = 0.
        
    def clr(self):
        cycle = np.floor(1+self.clr_iterations/(2*self.step_size))
        x = np.abs(self.clr_iterations/self.step_size - 2*cycle + 1)
        if self.scale_mode == 'cycle':
            return self.base_lr + (self.max_lr-self.base_lr)*np.maximum(0, (1-x))*self.scale_fn(cycle)
        else:
            return self.base_lr + (self.max_lr-self.base_lr)*np.maximum(0, (1-x))*self.scale_fn(self.clr_iterations)
        
    def on_train_begin(self, logs={}):
        logs = logs or {}

        if self.clr_iterations == 0:
            K.set_value(self.model.optimizer.lr, self.base_lr)
        else:
            K.set_value(self.model.optimizer.lr, self.clr())        
            
    def on_batch_end(self, epoch, logs=None):
        
        logs = logs or {}
        self.trn_iterations += 1
        self.clr_iterations += 1

        self.history.setdefault('lr', []).append(K.get_value(self.model.optimizer.lr))
        self.history.setdefault('iterations', []).append(self.trn_iterations)

        for k, v in logs.items():
            self.history.setdefault(k, []).append(v)
        
        K.set_value(self.model.optimizer.lr, self.clr())

In [ ]:
clr_triangular = CyclicLR(mode='triangular')

In [ ]:
EC.compile(optimizer=adamW,loss=tf.keras.losses.sparse_categorical_crossentropy)
EC.fit(x=[x,y],y=z,epochs=40,verbose=1,batch_size=4,validation_data=([encoder_test,decoder_inp_test],decoder_out_test),callbacks=[clr_triangular])

Epoch 1/40
492/492 [==============================] - 92s 102ms/step - loss: 7.9359 - val_loss: 8.0136
Epoch 2/40
492/492 [==============================] - 40s 81ms/step - loss: 8.0152 - val_loss: 8.0065
Epoch 3/40
492/492 [==============================] - 40s 81ms/step - loss: 6.9255 - val_loss: 8.0128
Epoch 4/40
492/492 [==============================] - 40s 82ms/step - loss: 8.0121 - val_loss: 8.0108
Epoch 5/40
492/492 [==============================] - 40s 81ms/step - loss: 8.0106 - val_loss: 8.0118
Epoch 6/40
492/492 [==============================] - 40s 81ms/step - loss: 8.0122 - val_loss: 8.0137
Epoch 7/40
492/492 [==============================] - 40s 82ms/step - loss: 8.0141 - val_loss: 8.0156
Epoch 8/40
492/492 [==============================] - 40s 82ms/step - loss: 8.0161 - val_loss: 8.0175
Epoch 9/40
492/492 [==============================] - 40s 82ms/step - loss: 8.0177 - val_loss: 8.0167
Epoch 10/40
492/492 [==============================] - 40s 81ms/step - loss: 8.01

In [ ]:
EC.compile(optimizer=tf.keras.optimizers.SGD(0.01),loss=tf.keras.losses.sparse_categorical_crossentropy)
EC.fit(x=[x,y],y=z,epochs=260,verbose=1,batch_size=4,validation_data=([encoder_test,decoder_inp_test],decoder_out_test),callbacks=[clr_triangular])

Epoch 1/260
492/492 [==============================] - 115s 90ms/step - loss: 7.9496 - val_loss: 7.6320
Epoch 2/260
492/492 [==============================] - 34s 68ms/step - loss: 7.2261 - val_loss: 3.6574
Epoch 3/260
492/492 [==============================] - 33s 66ms/step - loss: 3.5300 - val_loss: 3.5623
Epoch 4/260
492/492 [==============================] - 33s 66ms/step - loss: 3.4411 - val_loss: 3.4381
Epoch 5/260
492/492 [==============================] - 33s 67ms/step - loss: 3.3367 - val_loss: 3.3311
Epoch 6/260
492/492 [==============================] - 33s 66ms/step - loss: 3.2589 - val_loss: 3.2775
Epoch 7/260
492/492 [==============================] - 33s 66ms/step - loss: 3.2374 - val_loss: 3.2409
Epoch 8/260
492/492 [==============================] - 33s 66ms/step - loss: 3.2486 - val_loss: 3.2196
Epoch 9/260
492/492 [==============================] - 33s 66ms/step - loss: 3.1802 - val_loss: 3.1991
Epoch 10/260
492/492 [==============================] - 33s 66ms/step - 